# Tests for Portfolio VaR Functions

In [129]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
import plotly.express as px

# Import functions
import PortfolioVaR as pv
import InteractivePlots

### Asset Normal/ Undiversified VaR

**NOTE**: figure out later if backtesting here makes sense, it should be possible.

**NOTE 2**: since we already have the parametric VaR, a Portfolio-Normal VaR function is redundant.

In [130]:
# Set parameters
confidence_level = 0.99
holding_period = 1

In [131]:
# Define a small time series of positions (3 assets over 5 days)
position_data = pd.DataFrame({
    "Asset_A": [98, 102, 100, 100, 108, 100],
    "Asset_B": [167, 198, 154, 145, 206, 210],
    "Asset_C": [345, 303, 306, 287, 312, 310],
    "Asset D": [123, 145, 167, 189, 312, 345]
}, index=pd.date_range("2023-01-01", periods=6))

In [132]:
# Compute Diversified VaR
var_series = pv.var_asset_normal(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period,
    undiversified=False
)

# Compute Undiversified VaR
uvar_series = pv.var_asset_normal(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period,
    undiversified=True
)

# Compute Diversification Benefit
div_benefit = uvar_series - var_series

# Combine in a DataFrame
result = pd.DataFrame({
    "Diversified_VaR": var_series,
    "Undiversified_VaR": uvar_series,
    "Diversification_Benefit": div_benefit
})

# Print result
print(result)


            Diversified_VaR  Undiversified_VaR  Diversification_Benefit
2023-01-02       223.596448         259.578571                35.982122
2023-01-03       212.953838         246.464272                33.510434
2023-01-04       217.701814         249.581723                31.879909
2023-01-05       317.040603         355.614315                38.573712
2023-01-06       335.391955         374.027334                38.635379


In [133]:
# One-row static input
static_position = pd.DataFrame([[1_000_000, 2_000_000, 3_000_000]])

var = pv.var_asset_normal(static_position, confidence_level=0.99)
# -> Warning will be shown, and function returns None

/Users/aledo/Documents/GitHub/VaR/Functions/PortfolioVaR.py:46: UserWarning:

You must provide a time series of positions (at least 2 rows).



### Marginal VaR

In [134]:
# Call the marginal_var function
marginal_df = pv.marginal_var(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period
)

# Print the result
print(marginal_df)

             Asset_A   Asset_B   Asset_C   Asset D
2023-01-02  0.101070  0.515537  0.117163  0.522143
2023-01-03  0.100042  0.499514  0.124391  0.526711
2023-01-04  0.100901  0.497668  0.124423  0.527727
2023-01-05  0.103074  0.504505  0.120231  0.527143
2023-01-06  0.103232  0.503674  0.120409  0.527450


### Incremental, Component and Relative Component VaR

In [135]:
# Test Component VaR
component_df = pv.component_var(position_data, confidence_level, holding_period)
print("Component VaR:")
print(component_df)

# Test Relative Component VaR
relative_df = pv.relative_component_var(position_data, confidence_level, holding_period)
print("\nRelative Component VaR:")
print(relative_df)

# Test Incremental VaR (e.g., add 100 to Asset_B)
change_vector = [0, 100, 0, 0]
ivar_series = pv.incremental_var(position_data, change_vector, confidence_level, holding_period)
print("\nIncremental VaR (100 increase in Asset_B):")
print(ivar_series)

Component VaR:
              Asset_A     Asset_B    Asset_C     Asset D
2023-01-02  10.309160  102.076302  35.500290   75.710697
2023-01-03  10.004244   76.925094  38.063781   87.960719
2023-01-04  10.090100   72.161826  35.709442   99.740447
2023-01-05  11.131945  103.928083  37.512083  164.468492
2023-01-06  10.323154  105.771583  37.326919  181.970299

Relative Component VaR:
             Asset_A   Asset_B   Asset_C   Asset D
2023-01-02  0.046106  0.456520  0.158769  0.338604
2023-01-03  0.046978  0.361229  0.178742  0.413051
2023-01-04  0.046348  0.331471  0.164029  0.458152
2023-01-05  0.035112  0.327807  0.118319  0.518762
2023-01-06  0.030779  0.315367  0.111293  0.542560

Incremental VaR (100 increase in Asset_B):
2023-01-02    51.553688
2023-01-03    49.951360
2023-01-04    49.766776
2023-01-05    50.450526
2023-01-06    50.367421
Freq: D, dtype: float64


### Final Check

In [136]:
# 1. Get total VaR from your main function
var_series = pv.var_asset_normal(position_data, confidence_level)

# 2. Sum of Component VaR per day
component_df = pv.component_var(position_data, confidence_level)
component_sum = component_df.sum(axis=1)

# 3. Sum of Relative Component VaR per day
relative_df = pv.relative_component_var(position_data, confidence_level)
relative_sum = relative_df.sum(axis=1)

# 4. Build comparison table
check_df = pd.DataFrame({
    "Portfolio_VaR": var_series,
    "Sum_Component_VaR": component_sum,
    "Diff": var_series - component_sum,
    "Sum_Relative_CVaR": relative_sum
})

print(check_df)


            Portfolio_VaR  Sum_Component_VaR          Diff  Sum_Relative_CVaR
2023-01-02     223.596448         223.596448  0.000000e+00                1.0
2023-01-03     212.953838         212.953838  2.842171e-14                1.0
2023-01-04     217.701814         217.701814  2.842171e-14                1.0
2023-01-05     317.040603         317.040603  0.000000e+00                1.0
2023-01-06     335.391955         335.391955  5.684342e-14                1.0


### Visualizations

In [137]:
InteractivePlots.plot_var_series(var_series, uvar_series)

In [138]:
InteractivePlots.plot_risk_contribution_pie(component_df)

In [139]:
InteractivePlots.plot_risk_contribution_lines(component_df)

---
### Appendix

- **Asset-Normal VaR (AN VaR)**  
  $$
  \text{VaR}_t = z_\alpha \cdot \sqrt{x_t^\top \Sigma x_t} \cdot \sqrt{h}
  $$

- **Undiversified VaR (UVaR)**  
  $$
  \text{UVaR}_t = z_\alpha \cdot \sum_{i=1}^N \sigma_i x_{i,t} \cdot \sqrt{h}
  $$

- **Marginal VaR**  
  $$
  \Delta \text{VaR}_{i,t} = \text{VaR}_t \cdot \frac{(\Sigma x_t)_i}{x_t^\top \Sigma x_t}
  $$

- **Component VaR**  
  $$
  \text{CVaR}_{i,t} = x_{i,t} \cdot \Delta \text{VaR}_{i,t}
  $$

- **Relative Component VaR**  
  $$
  \text{RCVaR}_{i,t} = \frac{\text{CVaR}_{i,t}}{\text{VaR}_t}
  $$

- **Incremental VaR**  
  $$
  \text{IVaR}_t = \Delta \text{VaR}_t^\top \cdot a
  $$

---

### Function Dependencies

```text
var_asset_normal()
 └── marginal_var()
      ├── component_var()
      │     └── relative_component_var()
      └── incremental_var()
```

---

### Notes
- All risk measures are in **monetary terms**.
- Covariance is based on returns inferred from time series of holdings.
- Inputs: matrix of monetary holdings, confidence level $z_\alpha$, and horizon $h$.